In [2]:
import numpy as np
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.core.framework import graph_pb2
from model import palm_detection_model
from utils import define_fake_4_channels_graph, set_pretrained_weights, display_nodes

In [3]:
# Create 4-channels inputs layer with slice operation
fake_graph = define_fake_4_channels_graph(input_size=(256, 256, 4))

input_size = (256, 256, 3)
model = palm_detection_model(input_size)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 128, 128, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (None, 128, 128, 32) 320         re_lu[0][0]                      
_____________________________________

In [4]:
# Set pretrained weights from npy files downloaded using 'Netron'
set_pretrained_weights(model, weights_dir='pretrained_weights/')

[INFO] Set all pretrained weights


In [ ]:
# Replace input layer and concatenate graph def
sess = K.get_session()
output_names = [node.op.name for node in model.outputs]
frozen_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_names)

## check nodes
display_nodes(frozen_def.node[0:30])

In [ ]:
display_nodes(fake_graph.node)

In [ ]:
# change input name for fake 4-channels
frozen_def.node[4].input[0] = 'slicing_inputs/strided_slice'
# merge original graph with fake graph 
concat_nodes = fake_graph.node[0:] + frozen_def.node[1:]

## check again
display_nodes(concat_nodes[:30])

In [ ]:
display_nodes(concat_nodes[-5:])

In [ ]:
# Save new defined graph
out_path = "./mediapipe_models/palm_detection_4channels.pb"
concat_graph_def = graph_pb2.GraphDef()
concat_graph_def.node.extend(concat_nodes)

with tf.gfile.GFile(out_path, 'w') as f:
    f.write(concat_graph_def.SerializeToString())

In [ ]:
# Convert TFLite model from frozen graph
outPath = "./mediapipe_models/palm_detection_4channels.tflite"
frozen_graph_file = "./mediapipe_models/palm_detection_4channels.pb"
input_names = ['input_1']
output_names = ['regressors/concat', 'classificators/concat']
converter = tf.lite.TFLiteConverter.from_frozen_graph(frozen_graph_file, input_names, output_names)

tflite_model = converter.convert()
open(outPath, "wb").write(tflite_model)

In [ ]:
interpreter = 